In [2]:
import numpy as np
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# import new_merge.csv
df = pd.read_csv('new_merge.csv')

In [4]:
df.columns

Index(['date', 'event_count_battles', 'event_count_explosions',
       'event_count_violence', 'fatalities_battles', 'fatalities_explosions',
       'fatalities_violence', 'GOSIF_GPP', 'rainfall_chirps', 'GOSIF_GPP_SD',
       'elevation', 'soil', 'lon_x', 'lat_x', 'temperature_2m_mean',
       'temperature_2m_mean_sd', 'shortwave_radiation_sum',
       'shortwave_radiation_sum_sd', 'precipitation_sum',
       'precipitation_sum_sd', 'nearest_neighbor_distance_x', 'market_lat',
       'market_lon', 'price_index', 'estimated_population', 'title',
       'overall_phase', 'country', 'phase3_worse_population',
       'phase3_worse_percentage', 'phase1_population', 'phase1_percent',
       'phase2_population', 'phase2_percent', 'phase3_population',
       'phase3_percent', 'phase4_population', 'phase4_percent',
       'phase5_population', 'phase5_percent', 'phase3_worse_percentage_manual',
       'phase3_plus_phase4', 'phase2_worse_percentage_manual',
       'phase2_plus_phase3', 'phase2_pl

In [5]:
# rename lat_x lon_x to lat lon
df.rename(columns={'lat_x':'lat', 'lon_x':'lon'}, inplace=True)

In [6]:
# extract the locations and date
location = df[['date','lat','lon']]

In [7]:
# unique
location = location.drop_duplicates()

In [8]:
location

,date,lat,lon
0,2017-01-01,-5.533893,14.070903
1,2017-01-01,-5.493558,14.068317
2,2017-01-01,-5.289988,14.314904
3,2017-01-01,6.765411,15.778695
4,2017-01-01,7.248169,16.437358
...,...,...,...
38771,2022-12-01,37.037830,71.456211
38772,2022-12-01,35.171917,71.487215
38773,2022-12-01,34.752987,71.504960
38774,2022-12-01,34.753595,71.504927


In [10]:
lat_lon = location[['lat','lon']].drop_duplicates()

In [12]:
lat_lon = lat_lon.reset_index(drop=True)

In [13]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import tqdm
import time

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session



# create a new df to store the data
result = pd.DataFrame(columns=['date', 'lat', 'lon', 'soil', 'temperature_2m_mean_sd'])

# loop through each lat and lon
for i in tqdm.tqdm(range(len(lat_lon))):
    # request the data from the API
    url = 'https://archive-api.open-meteo.com/v1/archive?latitude=' + str(lat_lon['lat'][i]) + '&longitude=' + str(lat_lon['lon'][i]) + '&start_date=2017-01-01&end_date=2022-12-31&hourly=soil_moisture_0_to_7cm&models=best_match'
    try:
        response = requests_retry_session().get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as errh:
        print ("Http Error:",errh)
    except requests.exceptions.ConnectionError as errc:
        print ("Error Connecting:",errc)
    except requests.exceptions.Timeout as errt:
        print ("Timeout Error:",errt)
    except requests.exceptions.RequestException as err:
        print ("OOps: Something Else",err)
    data = response.json()
    # create a new df to store the data
    df = pd.DataFrame(columns=['date', 'lat', 'lon', 'soil_moisture_0_to_7cm'])
    # extract time and temperature_2m_mean from the json file
    df['date'] = data['hourly']['time']
    df['soil_moisture_0_to_7cm'] = data['hourly']['soil_moisture_0_to_7cm']
    df['lat'] = lat_lon['lat'][i]
    df['lon'] = lat_lon['lon'][i]
    # convert date to datetime
    df['date'] = pd.to_datetime(df['date'])
    # calculate montely average and sd
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['month_year'] = df['month'].astype(str) + '-' + df['year'].astype(str)
    df['month_year'] = pd.to_datetime(df['month_year'])
    df['month_year'] = df['month_year'].dt.strftime('%Y-%m')
    df = df.groupby(['month_year', 'lat', 'lon']).agg({'soil_moisture_0_to_7cm': ['mean', 'std']}).reset_index()
    df.columns = ['month_year', 'lat', 'lon', 'soil_moisture_0_to_7cm_mean', 'soil_moisture_0_to_7cm_sd']
    # rename month_year to date
    df = df.rename(columns={'month_year': 'date'})
    # concat the data
    result = pd.concat([result, df], axis=0)
    # sleep for 1 second
    time.sleep(1)



  0%|          | 0/1878 [00:00<?, ?it/s]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_71244\2656724782.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month_year'] = pd.to_datetime(df['month_year'])
  0%|          | 1/1878 [00:03<1:41:44,  3.25s/it]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_71244\2656724782.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month_year'] = pd.to_datetime(df['month_year'])
  0%|          | 2/1878 [00:05<1:31:04,  2.91s/it]C:\Users\WeilunShi\AppData\Local\Temp\ipykernel_71244\2656724782.py:60: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please sp

In [15]:
# drop temperature_2m_mean_sd
result = result.drop(columns=['temperature_2m_mean_sd'])
# drop soil
result = result.drop(columns=['soil'])

In [21]:
# determine whether date lon and lat are unique
result[['date', 'lat', 'lon']].duplicated().sum()

0

In [24]:
result

,date,lat,lon,soil_moisture_0_to_7cm_mean,soil_moisture_0_to_7cm_sd
0,2017-01,-5.533893,14.070903,0.451499,0.027810
1,2017-02,-5.533893,14.070903,0.449365,0.027560
2,2017-03,-5.533893,14.070903,0.441913,0.029305
3,2017-04,-5.533893,14.070903,0.465161,0.023311
4,2017-05,-5.533893,14.070903,0.423136,0.029051
...,...,...,...,...,...
67,2022-08,31.576524,70.151667,0.380845,0.040801
68,2022-09,31.576524,70.151667,0.324443,0.042739
69,2022-10,31.576524,70.151667,0.127489,0.042369
70,2022-11,31.576524,70.151667,0.274672,0.085663


In [25]:
# export to csv
result.to_csv('soil_moisture_0_to_7cm.csv', index=False)

In [26]:
# Step 1: Create a DataFrame to hold the missing data report
missing_data = pd.DataFrame()

# Step 2: Calculate the total number of missing values for each variable
missing_data['Total'] = result.isnull().sum()

# Step 3: Calculate the percentage of missing values for each variable
missing_data['Percent'] = result.isnull().mean() * 100

# Check the missing data report
print(missing_data)

missing_data.to_csv('missing_data_report_soil.csv')

                             Total  Percent
date                             0      0.0
lat                              0      0.0
lon                              0      0.0
soil_moisture_0_to_7cm_mean      0      0.0
soil_moisture_0_to_7cm_sd        0      0.0
